In [1]:
from ict_eeio.mrio import (
    get_all_industries,
    load_base,
    get_demand_categories,
    get_industry_output,
    get_monetary_flow,
    get_factor_input_per_Y_unit,
    get_leontief_flow,
)
from ict_eeio.eeio import (
    get_direct_impact_coefficient,
    get_cba_impact,
    get_pba_impact,
    get_total_direct_impact,
)

/Users/thibaultsimon/Documents/projects/thesis/1_eeioa/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
exio = load_base("2015")
i = "Manufacture of office machinery and computers (30)"

# Final demand

$$y_i = \sum_{d=0}Y_{id}$$


In [3]:
exio.Y.sum(axis=1)[("global", i)]

yi = 0
for demand_category in get_demand_categories(exio):
    yi += exio.Y["global", demand_category][("global", i)]
print(yi)
print(exio.Y.sum(axis=1)[("global", i)])

450618.71005212935
450618.71005212935


# Industry output


In [4]:
get_industry_output(exio, i)

exio.x

indout
region sector                                                          
global Cultivation of paddy rice                           2.649131e+05
       Cultivation of wheat                                2.198505e+05
       Cultivation of cereal grains nec                    3.355928e+05
       Cultivation of vegetables, fruit, nuts              1.040142e+06
       Cultivation of oil seeds                            1.968908e+05
...                                                                 ...
       Activities of membership organisation n.e.c. (91)   8.343581e+05
       Recreational, cultural and sporting activities ...  1.790953e+06
       Other service activities (93)                       1.229782e+06
       Private households with employed persons (95)       2.682935e+05
       Extra-territorial organizations and bodies          1.181399e-02

[163 rows x 1 columns]

$$x = Ze + Ye$$


In [5]:
exio.L.dot(exio.Y.sum(axis=1))[("global", i)]

960558.6979638322

Charpentier:

$$P_i = \sum{C_{ij} + D_i}$$

Us:

$$x_i = Y_i + \sum_{j=0}{Z_{ji}}$$


In [6]:
intermediate_demand = 0
for j in get_all_industries(exio):
    intermediate_demand += get_monetary_flow(exio, supplier=i, buyer=j)

final_demand = 0
for demand_category in get_demand_categories(exio):
    final_demand += exio.Y[("global", demand_category)][("global", i)]

intermediate_demand + final_demand

960558.6979637211

$$x = \sum_{i=0}{x_i}$$


In [7]:
x_total = 0

for industry in get_all_industries(exio):
    intermediate_demand = 0
    for j in get_all_industries(exio):
        intermediate_demand += get_monetary_flow(exio, supplier=industry, buyer=j)

    final_demand = 0
    for demand_category in get_demand_categories(exio):
        final_demand += exio.Y[("global", demand_category)][("global", industry)]

    x_total += intermediate_demand + final_demand

print(x_total)
print()
print(exio.x.sum(axis=0)["indout"])

135059649.52699032

135059649.52697563


On peut aussi faire non pas avec $Z$ mais avec $A$ et $x$ puisque

$$Z = Ax$$


In [8]:
intermediate_demand = 0

for j in get_all_industries(exio):
    intermediate_demand += get_factor_input_per_Y_unit(
        exio, supplier=i, buyer=j
    ) * get_industry_output(exio, industry=j)

intermediate_demand + final_demand

509939.98791159183

Charpentier:

$$C_{ij} = a_{ij} P_j$$

Us:

$$Z_{ij} = A_{ij} x_j$$

D'où

$$x_i = Y_i + \sum_{j=0}{A_{ij} x_j}$$

<!-- $$Z_{i} =  \sum_{j=0}{A_{ij} \times x_j}$$ -->


## Leontief


$$x = Ly$$


In [9]:
y = 0
for demand_category in get_demand_categories(exio):
    y += exio.Y[("global", demand_category)][("global", i)]

Z = 0
for j in get_all_industries(exio):
    Z += get_monetary_flow(exio, supplier=i, buyer=j)

print(Z + y)
print(exio.L.dot(exio.Y.sum(axis=1))[("global", i)])

960558.6979637211
960558.6979638322


Avec $$Ly$$ on peut séparer les émissions directes des émissions de la demande finale


In [10]:
exports = 0
sum_y = exio.Y.sum(axis=1)

for buyer in get_all_industries(exio):
    if buyer != i:
        exports += exio.L[("global", buyer)][("global", i)] * sum_y[("global", buyer)]

direct = exio.L[("global", i)][("global", i)] * sum_y[("global", i)]
print(direct)
print(exports)
print((direct + exports))
# print(exio.L.dot(exio.Y.sum(axis=1))[('global', i)])

621117.4687559718
339441.22920786036
960558.6979638322


# Impact


In [11]:
exio.impacts.F

region                                                                global  \
sector                                             Cultivation of paddy rice   
impact                                                                         
Value Added                                                     1.202696e+05   
Employment                                                      7.075690e+04   
Employment hour                                                 1.384040e+11   
GHG emissions (GWP100) | Problem oriented appro...              1.114999e+12   
Human toxicity (USEtox) | USEtox2008 | CTUh (Ro...              9.958195e+02   
...                                                                      ...   
Unused Domestic Extraction - Oil and Gas                        0.000000e+00   
Unused Domestic Extraction - Non-metalic Minerals               1.292324e+03   
Unused Domestic Extraction - Iron Ore                           0.000000e+00   
Unused Domestic Extraction - Non-ferous metal ores              0.000000e+00   
Land use Crop, Forest, Pasture                                  1.945323e+06   

region                                                                   \
sector                                             Cultivation of wheat   
impact                                                                    
Value Added                                                1.230329e+05   
Employment                                                 4.644899e+04   
Employment hour                                            9.964430e+10   
GHG emissions (GWP100) | Problem oriented appro...         2.746974e+11   
Human toxicity (USEtox) | USEtox2008 | CTUh (Ro...         1.076112e+03   
...                                                                 ...   
Unused Domestic Extraction - Oil and Gas                   1.808732e+02   
Unused Domestic Extraction - Non-metalic Minerals          5.321785e+02   
Unused Domestic Extraction - Iron Ore                      0.000000e+00   
Unused Domestic Extraction - Non-ferous metal ores         0.000000e+00   
Land use Crop, Forest, Pasture                             3.574680e+06   

region                                                                               \
sector                                             Cultivation of cereal grains nec   
impact                                                                                
Value Added                                                            1.973456e+05   
Employment                                                             4.930995e+04   
Employment hour                                                        1.025244e+11   
GHG emissions (GWP100) | Problem oriented appro...                     2.594728e+11   
Human toxicity (USEtox) | USEtox2008 | CTUh (Ro...                     7.756751e+02   
...                                                                             ...   
Unused Domestic Extraction - Oil and Gas                               7.780262e+01   
Unused Domestic Extraction - Non-metalic Minerals                      9.275696e+02   
Unused Domestic Extraction - Iron Ore                                  0.000000e+00   
Unused Domestic Extraction - Non-ferous metal ores                     0.000000e+00   
Land use Crop, Forest, Pasture                                         4.663673e+06   

region                                                                                     \
sector                                             Cultivation of vegetables, fruit, nuts   
impact                                                                                      
Value Added                                                                  7.664352e+05   
Employment                                                                   5.228367e+05   
Employment hour                                                              1.092905e+12   
GHG emissions (GWP100) | Problem oriented appro... 

Total direct impact is

$$F_i = x_i \times S_i$$


In [12]:
print(get_total_direct_impact(exio, i))
print(get_industry_output(exio, i) * get_direct_impact_coefficient(exio, i))

11419040378.450317
11419040378.45032


# PBA

Total direft impact $F$ equals PBA impact in a single region MRIO

For sector $i$, pba is the sum of productions impacts and impacts occuring during the final consumption.
As $x_i = \sum{Z_{ij} + Y_i}$

$$pba = (\sum_{j=0}{Z_{ji} + Y_i}) \times S_i$$

<!-- $$D_{pba}^i = \sum_{s=0}^m F^i_s + \sum_{c=0}^w F^i_{Yc}$$ -->


In [13]:
print(get_pba_impact(exio, i))

11419040378.453083


In [14]:
Ze = 0
for j in get_all_industries(exio):
    if j != i:  # On le compte à part
        Ze += get_monetary_flow(exio, supplier=i, buyer=j)

# Self est une partie de Z
self = get_monetary_flow(exio, supplier=i, buyer=i)

Ye = 0
for demand_category in get_demand_categories(exio):
    Ye += exio.Y[("global", demand_category)][("global", i)]

S = get_direct_impact_coefficient(base=exio, industry=i)

print(((Ze + self) + Ye) * S)

11419040378.451761


Using $Ly$ instead of $Z + Y$


In [15]:
exports = 0
sum_y = exio.Y.sum(axis=1)

for buyer in get_all_industries(exio):
    if buyer != i:
        exports += exio.L[("global", buyer)][("global", i)] * sum_y[("global", buyer)]

direct = exio.L[("global", i)][("global", i)] * sum_y[("global", i)]

print((direct + exports) * S)

11419040378.453083


# CBA

S _ L _ Y

$$
D_{cba}^i =  \sum_{j=0} {(L_{ij} \times S_j \times y_i )}
$$


In [16]:
print(get_cba_impact(exio, i))

234955075933.86017


In [17]:
Y = 0
for demand_category in get_demand_categories(exio):
    Y += exio.Y[("global", demand_category)][("global", i)]

total = 0
for j in get_all_industries(exio):
    s = get_direct_impact_coefficient(exio, j)
    l = get_leontief_flow(exio, buyer=i, supplier=j)
    total += s * l * Y

print(total)

234955075933.86017


# Compute CBA from PBA

## CBA

$cba = upstream + direct$

$cba = pba - (final consumption + exports) + upstream$

## PBA

$pba = direct + downstream$

$pba = cba - upstream + (final consumption + exports)$


In [18]:
# del final_consumption
# del exports
# del upstream

downstream = 0
sum_y = exio.Y.sum(axis=1)

for buyer in get_all_industries(exio):
    if buyer != i:
        downstream += (
            exio.L[("global", buyer)][("global", i)] * sum_y[("global", buyer)]
        )

direct = exio.L[("global", i)][("global", i)] * sum_y[("global", i)]

downstream *= S
direct *= S

upstream = 0
for supplier in get_all_industries(exio):
    if supplier != i:
        upstream += (
            exio.L[("global", i)][("global", supplier)]
            * sum_y[("global", i)]
            * get_direct_impact_coefficient(exio, supplier)
        )

print("PBA")
print(get_cba_impact(exio, i) - upstream + downstream)
print(get_pba_impact(exio, i))
print(direct + downstream)

print()

print("CBA")
print(get_pba_impact(exio, i) - downstream + upstream)
print(get_cba_impact(exio, i))
print(upstream + direct)

PBA
11419040378.453136
11419040378.453083
11419040378.453083

CBA
234955075933.8601
234955075933.86017
234955075933.8601


# .dot Ly

Ici on refait à la main Ly pour pouvoir le désaggréger


In [19]:
# Refaire le .dot à la main pour tracer la production
total = {}
sum_y = exio.Y.sum(axis=1)

# On fait le total de toutes les colonnes ligne par ligne
for buyer in get_all_industries(exio):
    total[buyer] = 0
    for supplier in get_all_industries(exio):
        total[buyer] += (
            exio.L[("global", supplier)][("global", buyer)]
            * sum_y[("global", supplier)]
        )

print(total[i])
print(exio.L.dot(exio.Y.sum(axis=1))[("global", i)])

960558.6979638322
960558.6979638322


In [20]:
# Now we apply the same strategy for a single industry
exports = 0
sum_y = exio.Y.sum(axis=1)

for buyer in get_all_industries(exio):
    if buyer != i:
        exports += exio.L[("global", buyer)][("global", i)] * sum_y[("global", buyer)]

direct = exio.L[("global", i)][("global", i)] * sum_y[("global", i)]
print(direct)
print(exports)
print((direct + exports))
print(exio.L.dot(exio.Y.sum(axis=1))[("global", i)])

621117.4687559718
339441.22920786036
960558.6979638322
960558.6979638322
